<a href="https://colab.research.google.com/github/nagarajan1/STTP-SLOT-1/blob/master/word2vecCNNsentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.models import KeyedVectors
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
import codecs


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
from keras.callbacks import TensorBoard
import os


def make_tensorboard(set_dir_name='',
                     embeddings_freq=0,
                     embeddings_layer_names=None,
                     embeddings_metadata=None):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir,
                              embeddings_freq=embeddings_freq,
                              embeddings_layer_names=embeddings_layer_names,
                              embeddings_metadata=embeddings_metadata)
    return tensorboard, log_dir

In [ ]:
#from make_tensorboard import make_tensorboard

ModuleNotFoundError: ignored

In [ ]:
np.random.seed(42)

INPUT_FILE = "drive/My Drive/training.txt"
# You have to get the model
#    https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download
WORD2VEC_MODEL = "drive/My Drive/GoogleNews-vectors-negative300.bin.gz"
VOCAB_SIZE = 5000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 10

In [ ]:
counter = collections.Counter()
fin = codecs.open(INPUT_FILE, "r", encoding='utf-8')
maxlen = 0
import nltk
nltk.download('punkt')
for line in fin:
    _, sent = line.strip().split("\t")
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1
fin.close()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

xs, ys = [], []
fin = codecs.open(INPUT_FILE, "r", encoding='utf-8')
for line in fin:
    label, sent = line.strip().split("\t")
    ys.append(int(label))
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    wids = [word2index[word] for word in words]
    xs.append(wids)
fin.close()

In [ ]:
X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = \
    train_test_split(X, Y, test_size=0.3, random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)


(4960, 42) (2126, 42) (4960, 2) (2126, 2)


In [ ]:
# load word2vec model
word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary=True)
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen,
                    weights=[embedding_weights],
                    trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS,
                 activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])
tensorboard, log_dir = make_tensorboard(
    set_dir_name='keras_finetune_word2vec_embeddings')

history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[tensorboard],
                    validation_data=(Xtest, Ytest))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4960 samples, validate on 2126 samples
Epoch 1/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.1253 - accuracy: 0.9532 - val_loss: 0.0229 - val_accuracy: 0.9934
Epoch 2/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.0113 - accuracy: 0.9972 - val_loss: 0.0155 - val_accuracy: 0.9958
Epoch 3/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.0048 - accuracy: 0.9990 - val_loss: 0.0140 - val_accuracy: 0.9939
Epoch 4/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 0.0141 - val_accuracy: 0.9962
Epoch 5/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.0146 - val_accuracy: 0.9929
Epoch 6/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.0026 - accuracy: 0.9996 - val_loss: 0.0137 - val_accuracy: 0.9944
Epoch 7/10
4960/4960 [==============================] - 10s 2ms/step - loss: 0.001

In [ ]:
# evaluate model
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

2126/2126 [==============================] - 1s 370us/step
Test score: 0.014, accuracy: 0.995
